# Coordinate systems as operators 

## Cartesian

In [ ]:
import numpy as np
from numpy import pi 
from kingdon import Algebra
exp = lambda x: x.exp()
from itertools import pairwise,chain
c = [0, 1810039, 14245634, 7696563, 15149450, 6727198, 15117058, 10909213, 6710886] 

pga = Algebra(2,0,1,start_index=0) 
locals().update(pga.blades)

T     = lambda x,y: exp(-(x*e01 + y*e02)/2)  # translation operator 
dists = np.linspace(-1,1,11)   # parameters
o     = e0.dual() # origin 
 
 # Create points by moving origin around 
points   = [T(x,y)>>o for x in dists for y in dists]

# create grid by making line-segments connecting points (tuples in ganja)
x_lines = [pairwise( [T(x,y)>>o for y in dists]) for x in dists]
y_lines = [pairwise( [T(x,y)>>o for x in dists]) for y in dists]

pga.graph(
    c[0], *points,
    c[1], *list(chain(*x_lines)),
    c[2], *list(chain(*y_lines)),
    grid=False, lineWidth=3, 
)

## Polar

In [2]:
pga = Algebra(2,0,1,start_index=0) 
locals().update(pga.blades) 

# operators
T  = lambda x,y: exp(-(x*e01 + y*e02)/2) 
R  = lambda theta: exp(-theta*e12/2)
RT = lambda rho,theta: R(theta)*T(rho,0) 
 
# parameters
angles   = np.linspace(-pi,pi,41)
dists    = np.linspace(.1,2,11)

o = e0.dual() # origin 

 # Create points by moving origin around 
points  = [RT(rho, theta)>>o for theta in angles for rho in dists]
# create grid by connecting points 
angular_lines = [pairwise( [RT(rho, theta)>>o for theta in angles]) for rho in dists]
radial_lines  = [pairwise( [RT(rho, theta)>>o for rho in dists]) for theta in angles]

pga.graph(
    c[0], *points,
    c[1], *list(chain(*angular_lines)),
    c[2], *list(chain(*radial_lines)),
    grid=False, lineWidth=3, 
)

### Tangent Frame 
Since we have operationalized the coordinate function, computing a tangent frame is simple. 

In [3]:
# create a tangent frame at each point (vectors are tuples of 2 points)
tmag = .1 # tangent vector magnitude
radial   = (o, T(tmag,0)>>o) # tangent vector in radial direction
angular  = (o, T(0,tmag)>>o) # tangent vector in angular direction
radials  = [RT(rho, theta)>>radial for theta in angles for rho in dists]
angulars = [RT(rho, theta)>>angular for theta in angles for rho in dists]


pga.graph(
    c[0], *points,
    c[1], *radials,
    c[2], *angulars,
    grid=False, lineWidth=4, )

## 3d Polar ( Latitude, longitude)


In [19]:
import numpy as np
from numpy import pi 
from kingdon import Algebra
exp = lambda x: x.exp()
from itertools import pairwise,chain

c = [0, 1810039, 14245634, 7696563, 15149450, 6727198, 15117058, 10909213, 6710886] 

pga = Algebra(3,0,1,start_index=0) 
locals().update(pga.blades)

# operators
T  = lambda x,y,z: exp(-(x*e01 + y*e02 +z*e03)/2) 
R  = lambda theta, phi : exp(-theta*e12/2) * exp(-phi*e13/2) 
RT = lambda rho,theta,phi: R(theta,phi)*T(rho,0,0) 

# parameters
thetas = np.linspace(-pi,pi,21)  
phis   = np.linspace(-pi/2,pi/2,11)[1:-1]  
rho    = 1 # earth radius

o           = e0.dual() # origin 
null_island = T(rho,0,0)>>o # lat=lon=0 point 
points      = [R(theta, phi)>>null_island for theta in thetas for phi in phis ]

# create grid by connecting points 
lat_lines = [pairwise( [R(theta,phi)>>null_island for theta in thetas]) for phi in phis]
lon_lines = [pairwise( [R(theta,phi)>>null_island for phi in phis]) for theta in thetas]

pga.graph(
    c[0], *points,
    c[5], null_island,
    c[1], *list(chain(*lat_lines)),
    c[2], *list(chain(*lon_lines)),
    grid=False, lineWidth=3,
)

### Tangent Frame 

In [23]:
import numpy as np
from numpy import pi 
from kingdon import Algebra
#exp = lambda x: x.exp()
from kingdon.numerical import exp # work around https://github.com/tBuLi/kingdon/issues/109


c = [0, 1810039, 14245634, 7696563, 15149450, 6727198, 15117058, 10909213, 6710886] 

pga = Algebra(3,0,1,start_index=0) 
locals().update(pga.blades)

# operators
T  = lambda x,y,z: exp(-(x*e01 + y*e02 +z*e03)/2) 
R  = lambda theta, phi : exp(-theta*e12/2)*exp(-phi*e13/2) 

# parameters
thetas = np.linspace(-pi,pi,21)              
phis   = np.linspace(-pi/2,pi/2,11)[1:-1]  
theta, phi = np.meshgrid(thetas,phis)
rho    = 2 

# objects
o     = e0.dual() # origin 
tmag  = rho/10 # magnitude of tangent vectors
up    = (o, T(tmag,0,0)>>o)
east  = (o, T(0,tmag,0)>>o)
north = (o, T(0,0,tmag)>>o)

# propgate tangent frame to all points
Rs     = R(theta,phi)*T(rho,0,0)  # you could set rho=earth_radius+altitude 
points = Rs>>o
easts  = Rs>>east  
norths = Rs>>north
ups    = Rs>>up

# helper to unzip tuples of arrays
format_for_ganja = lambda x: list(zip( *( [k.flatten() for k in x] ) )) 
easts, norths, ups = map(format_for_ganja, (easts, norths, ups))

pga.graph(
    c[0], *points.flatten(),
    c[1], *easts,
    c[2], *norths,
    c[3], *ups,
    grid=False, lineWidth=5,
)

## Toriodal

In [24]:
import numpy as np
from numpy import pi 
from kingdon import Algebra
from kingdon.numerical import exp 

c = [0, 1810039, 14245634, 7696563, 15149450, 6727198, 15117058, 10909213, 6710886] 

pga = Algebra(3,0,1,start_index=0) 
locals().update(pga.blades)

# operators
T  = lambda x,y,z: exp(-(x*e01 + y*e02 +z*e03)/2) 
R  = lambda theta, phi : exp(-theta*e12/2)*exp(-phi*e13/2) 

# parameters
thetas = np.linspace(-pi,pi,21)
phis   = np.linspace(-pi,pi,21)
r1,r2    = .5,1

# objects
o        = e0.dual() # origin 
tmag     = .1
normal   = (o, T(tmag,0,0)>>o)
poloidal = (o, T(0,tmag,0)>>o)
toroidal = (o, T(0,0,tmag)>>o)

# propgate tangent frame to all points
Rs = [ R(0,theta)*T(r2,0,0)*R(phi,0)*T(r1,0,0) for theta in thetas for phi in phis]
points     = [k>>o for k in Rs]
normals    = [k>>normal for k in Rs]
poloidals  = [k>>poloidal for k in Rs]
toroidals  = [k>>toroidal for k in Rs]

pga.graph(
    c[0], *points ,
    c[1], *poloidals,
    c[2], *toroidals,
    c[3], *normals,
    grid=False, lineWidth=4,
)